Librerias

In [ ]:
#LIBRERIAS DEL PROYECTO

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

#Este es driver que va a navegar.
driver = webdriver.Chrome()

Iniciar sesión

In [ ]:
# Función para iniciar sesión
arancia_url = 'http://arancia-001-site1.btempurl.com/'

driver.get(arancia_url)
time.sleep(3)

# Entrar a la Web
element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

# Colocar las claves
element = driver.find_element(By.ID, 'TextBox1')
element.send_keys('FRANCESC')
element = driver.find_element(By.ID, 'TextBox2')
element.send_keys('n4r4nj4720')

# click en entrar
element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

# clickear en la pestaña de "facturación"
element = driver.find_element(By.ID,"Button10")
element.click()
time.sleep(3)

# clickear en la sub-pestaña de "facturación" (para cargar facturas de ventas)
driver.switch_to.frame('facturacion')
element = driver.find_element(By.ID,"Button5")
element.click()
time.sleep(1)

Cambiar de frame al marco interno

In [33]:
driver.switch_to.frame('marco')

Establecer factura manual

In [34]:
# Quitar check de "aéreo"
element = driver.find_element(By.ID,"CheckBoxList1_0")
element.click()
time.sleep(2)

# Poner check en "manual"
element = driver.find_element(By.ID,"CheckBoxList1_3")
element.click()
time.sleep(2)

# Buscar el input de IVAS
element = driver.find_element(By.ID, "contar")

# Seleccionar todo el texto actual
element.send_keys(Keys.CONTROL, "a")
time.sleep(0.2)

# Escribir el nuevo valor (3 por la información que manejamos)
element.send_keys("3")

# Disparar el evento de cambio usando TAB (como si el usuario saliera del campo)
element.send_keys(Keys.TAB)
time.sleep(2)

CAMBIAR LOS IVAS

In [41]:
# Valores específicos para cada campo
valores = {
    "ivasa1": "0",
    "ivasa2": "10.5"  # con coma, si la página lo acepta así
}

for field_id, valor in valores.items():
    element = driver.find_element(By.ID, field_id)

    # Seleccionar todo el texto actual
    for i in range(4):
        element.send_keys(Keys.BACKSPACE)
    time.sleep(0.2)

    # Escribir directamente el nuevo valor (sin dejar el campo vacío)
    element.send_keys(valor)
    time.sleep(3)

Hasta aquí logramos que el sistema configure todo para empezar a cargar facturas....
Para que funcione, creo que debemos hacer lo siguiente:
- pasarle la lista de facturas que queremos que cargue + el csv de lo que ya existe
- que haga una comparación con las facturas que existen actualmente
- que las cargue. 

Para una primera versión, podemos hacer la comparación manual tal vez.

De aquí hacia abajo no funciona la cosa

----------------------



PASO FALTANTE: PROCESAR FACTURAS PARA OBTENER DATOS

In [18]:
import pandas as pd
import string
import numpy as np

data = pd.read_excel(r'C:\Users\ThinkPad-PC\Downloads\Mis Comprobantes Recibidos - CUIT 30714894346.xlsx', header=0, skiprows=1)

#TIPO DE FACTURA
data['Tipo'] = data['Tipo'].astype('string').str[-9:]
data['Tipo2'] = data['Tipo'].str[:7]
data['Tipo3'] = data['Tipo'].str[-1:]

# FACTURA
data['Punto de Venta'] = data['Punto de Venta'].astype(str).str.zfill(5)
data['Número Desde'] = data['Número Desde'].astype(str).str.zfill(8)
data['Factura'] = data['Punto de Venta'] + '-' + data['Número Desde']

# PROVEEDOR
data['Denominación Emisor'] = data['Denominación Emisor'].str.translate(
    str.maketrans('', '', string.punctuation))
data['Proveedor'] = data['Denominación Emisor'].str[:35]

# CUIT
data['CUIT'] = data['Nro. Doc. Emisor'].astype(str)


# IMPORTES
data['NETO 10.5'] = data['Neto Grav. IVA 10,5%'] 
data['IVA 10.5'] = data['IVA 10,5%'] 
data['NETO 21'] = data['Neto Grav. IVA 21%'] 
data['IVA 21'] = data['IVA 21%'] 
data['NO GRAVADO'] = data['Neto No Gravado'] 
data['EXENTO'] = data['Op. Exentas'] 
data['IMPUESTOS'] = data['Otros Tributos']
data['NETO 0'] = data['Neto Grav. IVA 0%']
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])

# PONER 0 A TODAS LAS COLUMNAS
columnas = ['NETO 0', 'NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS',
            'IVA 2,5%', 'IVA 5%', 'IVA 27%', 'TOTAL_NO_GRAVADO'] 
for col in columnas: 
    data[col] = data[col].fillna(0)

# TIPO DE CAMBIO ESTE CAMBIAR PORQUE CAMBIA A FUTURO
columnas_tc = ['NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS'] 
for col in columnas_tc: 
    data[col] = data[col] * data['Tipo Cambio']

# SUMAR LO QUE NO LLEVA IVA
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])

# Convertimos las columnas a str
data['NETO 10.5'] = data['NETO 10.5'].astype(str)
data['IVA 10.5'] = data['IVA 10.5'].astype(str)
data['NETO 21'] = data['NETO 21'].astype(str)
data['IVA 21'] = data['IVA 21'].astype(str)

# PONER EN NEGATICO LAS COLUMNAS QUE SON CREDITO
for col in columnas:
    data.loc[data['Tipo2'] == 'Crédito', col] = -data[col].astype(float)
    data[col] = data[col].astype(str)

# CREAR Y APLICAR MÁSCARA
clean = ['Fecha','Tipo2','Tipo3', 'Factura',
        'Proveedor','CUIT', 'NETO 10.5', 
        'NETO 21', 'IVA 10.5', 'IVA 21', 
        'TOTAL_NO_GRAVADO']

clean_data = data[clean]
clean_data

,Fecha,Tipo2,Tipo3,Factura,Proveedor,CUIT,NETO 10.5,NETO 21,IVA 10.5,IVA 21,TOTAL_NO_GRAVADO
0,03/10/2025,Factura,A,00008-00089548,CARNIVAL SRL,30676579938,3917.14,0.0,411.3,0.0,45431.56
1,03/10/2025,Factura,A,00008-00089801,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,280140.0
2,03/10/2025,Factura,A,00008-00089812,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,2221878.5
3,03/10/2025,Factura,A,00008-00089813,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,1721744.5000000002
4,04/10/2025,Factura,A,00008-00089862,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,116000.0
...,...,...,...,...,...,...,...,...,...,...,...
98,12/10/2025,Factura,A,00008-00091192,CARNIVAL SRL,30676579938,1226866.1,0.0,128820.94,0.0,26247.6
99,12/10/2025,Crédito,A,00008-00007783,CARNIVAL SRL,30676579938,-886097.05,-0.0,-93040.19,-0.0,-17378.8
100,12/10/2025,Crédito,A,00008-00007784,CARNIVAL SRL,30676579938,-886097.05,-0.0,-93040.19,-0.0,-17378.8
101,12/10/2025,Crédito,A,00008-00007781,CARNIVAL SRL,30676579938,-636399.52,-0.0,-66821.95,-0.0,-14317.72


In [20]:

for index, row in clean_data.iterrows():
    # Encuentra cada campo por su id y llena el valor
    driver.find_element(By.ID, 'DetailsView1_TextBox1').send_keys(row['Fecha'])
    driver.find_element(By.ID, 'DetailsView1_TextBox2').send_keys(row['Fecha'])
    driver.find_element(By.NAME, 'DetailsView1$ctl02').send_keys(row['Tipo3'])
    driver.find_element(By.NAME, 'DetailsView1$ctl03').send_keys(row['Factura'])
    driver.find_element(By.NAME, 'DetailsView1$ctl04').send_keys(row['Proveedor'])
    driver.find_element(By.NAME, 'DetailsView1$ctl05').send_keys(row['CUIT'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox3').send_keys(row['NETO 10.5'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox4').send_keys(row['NETO 21'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox5').send_keys(row['IVA 10.5'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox6').send_keys(row['IVA 21'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox7').send_keys(row['TOTAL_NO_GRAVADO'])
    # Haz clic en el botón "Agregar"
    driver.find_element(By.LINK_TEXT, 'Agregar').click()
    # Espera un poco para que la página se actualice
    time.sleep(3)

In [21]:
os.remove(r'C:\Users\ThinkPad-PC\Downloads\Mis Comprobantes Recibidos - CUIT 30714894346.xlsx')